# FEMA's National Flood Insurance Policy Database Data Exploration

### Load Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from pyspark.ml.stat import Correlation
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Imputer
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, ByteType, LongType, FloatType, ShortType
from pyspark.sql.functions import col, sum as spark_sum

### Initialize Spark Session

In [2]:
spark = SparkSession.builder \
    .config("spark.driver.memory", "10g") \
	.config("spark.executor.memory", "2g") \
    .config('spark.executor.instances', 5) \
	.config("spark.sql.debug.maxToStringFields", "100")\
    .appName("Flood Data") \
	.getOrCreate()

24/05/24 13:30:37 WARN Utils: Your hostname, Joyces-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.0.0.233 instead (on interface en0)
24/05/24 13:30:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/24 13:30:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Load data to Spark dataframe; Infer Schema

In [3]:
#path to data 
data_path = "./NFIP/nfip-flood-policies.csv"

df = spark.read.option("header","true").option("inferSchema","true").csv(data_path)

## Perform the Data Exploration
Below is the outline of the Data Exploration Section:
- Print Schema
- Column Description
- Display the Number of Variables (Columnns)
- Show the First Few Rows
- Display Number of Observations
- Missing Values in the Dataframe
- Statistics Summary and Data Distribution
- Correlations Among Variables
- Scatterplots

### Print Schema

Understanding the structure of data (column names and types) is crucial.

In [4]:
df.printSchema()

root
 |-- agriculturestructureindicator: string (nullable = true)
 |-- basefloodelevation: double (nullable = true)
 |-- basementenclosurecrawlspacetype: integer (nullable = true)
 |-- cancellationdateoffloodpolicy: date (nullable = true)
 |-- censustract: long (nullable = true)
 |-- condominiumindicator: string (nullable = true)
 |-- construction: string (nullable = true)
 |-- countycode: integer (nullable = true)
 |-- crsdiscount: double (nullable = true)
 |-- deductibleamountinbuildingcoverage: string (nullable = true)
 |-- deductibleamountincontentscoverage: string (nullable = true)
 |-- elevatedbuildingindicator: string (nullable = true)
 |-- elevationcertificateindicator: string (nullable = true)
 |-- elevationdifference: integer (nullable = true)
 |-- federalpolicyfee: integer (nullable = true)
 |-- floodzone: string (nullable = true)
 |-- hfiaasurcharge: integer (nullable = true)
 |-- houseofworshipindicator: string (nullable = true)
 |-- latitude: double (nullable = true)
 |--

### Column Description

**Description of the columns from the FEMA's National Flood Insurance Policy Database, grouped by their data types and purpose:**

**Geographic and Location Details**
- **censustract (long):** Census tract number indicating the specific area where the property is located, used for demographic analysis.
- **countycode (integer):** Numeric code representing the county in which the property is insured.
- **floodzone (string):** Designation of the flood zone according to FEMA's mapping, crucial for assessing the property's flood risk.
- **latitude (double), longitude (double):** Geographic coordinates specifying the precise location of the property.
- **propertystate (string):** The U.S. state where the property is located.
- **reportedcity (string):** The city reported for the insured property.
- **reportedzipcode (integer):** Zip code where the property is situated, used for localizing insurance coverage and risk.

**Property and Construction Details**
- **agriculturestructureindicator (string):** Indicates whether the property is used for agricultural purposes.
- **basementenclosurecrawlspacetype (integer):** Type of basement or crawlspace present at the property, affecting flood risk assessment.
- **construction (string):** Describes the type of construction materials and methods used, which can affect the property's vulnerability to flood damage.
- **numberoffloorsininsuredbuilding (integer):** Total floors in the insured building, important for determining potential flood damage and insurance coverage needs.
- **elevatedbuildingindicator (string):** Indicates whether the building is elevated, a key factor in reducing flood risk.

**Policy Details**
- **policycost (integer):** The total cost of the flood insurance policy.
- **policycount (integer):** The number of policies associated with a single property or account.
- **policyeffectivedate (date), policyterminationdate (date):** Start and end dates of the flood insurance coverage.
- **totalbuildinginsurancecoverage (integer), totalcontentsinsurancecoverage (integer):** The amount of insurance coverage for the building and its contents, respectively.
- **totalinsurancepremiumofthepolicy (integer):** Total premium amount for the flood insurance policy.

**Flood Risk Assessment Specifics**
- **basefloodelevation (double):** The base flood elevation expected for a particular area, critical for understanding flood risk levels.
- **elevationcertificateindicator (string), elevationdifference (integer):** Presence of an elevation certificate and the difference in elevation, respectively, both crucial for assessing compliance with floodplain management.
- **lowestadjacentgrade (double), lowestfloorelevation (double):** Measures of elevation that help determine the property's flood exposure.

**Insurance Policy Features**
- **crsdiscount (double):** Community Rating System discount applied to the policy, which can reduce insurance premiums based on community flood preparedness.
- **deductibleamountinbuildingcoverage (integer), deductibleamountincontentscoverage (integer):** Deductible amounts for building and contents coverage, influencing out-of-pocket costs after a flood.
- **hfiaasurcharge (integer):** Surcharge applied under the Homeowner Flood Insurance Affordability Act.
- **federalpolicyfee (integer):** A fee associated with the federal policy governing flood insurance.

**Special Indicators**
- **condominiumindicator (string), primaryresidenceindicator (string):** Indicate whether the insured property is a condominium or the primary residence of the owner.
- **houseofworshipindicator (string), nonprofitindicator (string):** Indicators of whether the property is used as a house of worship or is owned by a nonprofit organization, affecting policy terms and possibly qualifying for special considerations.
- **postfirmconstructionindicator (string):** Indicates if the building was constructed after the community's first Flood Insurance Rate Map was issued, which can affect insurance rates.
- **smallbusinessindicatorbuilding (string):** Indicates whether the insured building is used for small business purposes.

**Additional Policy and Coverage Information**
- **originalconstructiondate (date), originalnbdate (date):** Dates of original construction and the building's initial notebook entry, important for historical property assessments.
- **cancellationdateoffloodpolicy (date):** Date when the flood policy was cancelled, if applicable.
- **regularemergencyprogramindicator (string):** Indicates the type of FEMA program under which the policy is covered, distinguishing between regular and emergency management programs.
- **ratemethod (integer):** Describes the method used to calculate the insurance rate, impacting how premiums are determined.
- **locationofcontents (string):** Specifies where within the property the insured contents are located, relevant for claims and risk assessments.

### Display number of Variables (Columns)


In [5]:
num_variables = len(df.columns)
print("Number of Columns:", num_variables)

Number of Columns: 45


### Show the First Few Rows

In [6]:
df.show(5,vertical=True)

-RECORD 0--------------------------------------------------
 agriculturestructureindicator      | NULL                 
 basefloodelevation                 | NULL                 
 basementenclosurecrawlspacetype    | 2                    
 cancellationdateoffloodpolicy      | NULL                 
 censustract                        | 33013038500          
 condominiumindicator               | N                    
 construction                       | N                    
 countycode                         | 33013                
 crsdiscount                        | 0.0                  
 deductibleamountinbuildingcoverage | 0                    
 deductibleamountincontentscoverage | 0                    
 elevatedbuildingindicator          | N                    
 elevationcertificateindicator      | NULL                 
 elevationdifference                | 999                  
 federalpolicyfee                   | 13                   
 floodzone                          | X 

### Display Number of Observations

In [7]:
df.count()

50406943

FEMA's National Flood Insurance Policy Database, containing over 50 million (50,406,943) policy transactions.

### Missing Values in the Dataframe

In [8]:
#find number of missing values in the dataframe
missing_vals = df.select(*(spark_sum(col(i).isNull().cast("int")).alias(i) for i in df.columns))

In [9]:
missing_vals.show(vertical= True)

-RECORD 0--------------------------------------
 agriculturestructureindicator      | 38923313 
 basefloodelevation                 | 33636759 
 basementenclosurecrawlspacetype    | 802      
 cancellationdateoffloodpolicy      | 43614057 
 censustract                        | 467119   
 condominiumindicator               | 6        
 construction                       | 13       
 countycode                         | 48999    
 crsdiscount                        | 0        
 deductibleamountinbuildingcoverage | 661993   
 deductibleamountincontentscoverage | 5561584  
 elevatedbuildingindicator          | 258      
 elevationcertificateindicator      | 32606397 
 elevationdifference                | 0        
 federalpolicyfee                   | 0        
 floodzone                          | 169145   
 hfiaasurcharge                     | 0        
 houseofworshipindicator            | 34476251 
 latitude                           | 338699   
 locationofcontents                 | 15

**Here's a description of the missing value situation in the FEMA Flood Insurance Policy Database:** 

1. **High Missing Values:**
- **Base Flood Elevation, Latitude, Longitude, Lowest Adjacent Grade, Lowest Floor Elevation:** These fields each have around 50,406,943 missing values. This suggests a significant lack of geographic and elevation data, which are critical in flood insurance calculations.
- **Elevation Certificate Indicator, Elevation Difference:** Both fields are missing approximately 32,806,397 and 32,897,994 values respectively, indicating that elevation certificates, which are vital for verifying compliance with floodplain management regulations, are largely absent.
- **Obstruction Type:** Missing around 40,793,526 values, indicating that details about obstructions which can affect flood risk assessments are predominantly not reported.
- **House of Worship Indicator, Nonprofit Indicator:** Each has over 34,476,251 and 34,493,094 missing entries respectively, indicating a lack of identification of these property types, which might have different considerations in policy terms.

2. **Moderate Missing Values:**
- **Deductible Amount in Building Coverage, Deductible Amount in Contents Coverage:** Missing 15,649,149 and 18,265,104 values respectively, which implies incomplete data on policy deductibles that could affect premium calculations and risk assessments.
- **Location of Contents:** With 15,389,767 missing entries, there's substantial missing information on where contents are located within the insured buildings, which is vital for damage assessments.

3. **Low Missing Values:**
- **Census Tract, Flood Zone:** Missing 467,119 and 169,145 entries respectively. Although relatively lower, these still represent significant gaps, especially as these fields are crucial for location-specific risk assessment.
- **Number of Floors in Insured Building:** Missing data on 162,301 entries could affect understanding building structure and associated risk.

4. **Minimal to No Missing Values:**
- Fields like **CRS Discount, Federal Policy Fee**, and various policy-related dates (effectiveness, termination) and costs show zero missing values, indicating complete data in terms of policy transaction details.
- Similarly, **County Code, Construction, Condominium Indicator, Occupancy Type** show minimal missing data (under 50,000), suggesting good coverage of basic property and policyholder information.

Overall, the dataset shows a strong presence of policy and basic property information but suffers from a significant absence of detailed geographic and structural data. This gap in data can hamper effective risk assessment and pricing of flood insurance policies, especially in areas prone to flooding where such data is most critical. Addressing these missing values, either by data imputation where appropriate or by collecting missing data, could significantly enhance the robustness of any analysis or predictive modeling based on this dataset.

### Statistics Summary and Data Distribution

In [10]:
numerical_columns = [col_name for col_name, data_type in df.dtypes if data_type in ["int", "bigint", "double"]]

# Select only numerical columns
numerical_df = df.select(*numerical_columns)

# Generate summary statistics
summary_stats = numerical_df.describe()

# Show summary statistics
summary_stats.show(vertical = True)

24/05/24 13:33:40 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


-RECORD 0------------------------------------------------
 summary                          | count                
 basefloodelevation               | 16770184             
 basementenclosurecrawlspacetype  | 50406141             
 censustract                      | 49939824             
 countycode                       | 50357944             
 crsdiscount                      | 50406943             
 elevationdifference              | 50406943             
 federalpolicyfee                 | 50406943             
 hfiaasurcharge                   | 50406943             
 latitude                         | 50068244             
 longitude                        | 50068244             
 lowestadjacentgrade              | 15466364             
 lowestfloorelevation             | 17346341             
 numberoffloorsininsuredbuilding  | 50244642             
 occupancytype                    | 50406937             
 policycost                       | 50406943             
 policycount  

The summary statistics for the FEMA National Flood Insurance Policy Database provide a comprehensive overview of various policy and property-related numerical attributes. These statistics include measures of central tendency, dispersion, and range, all of which are critical for understanding the distribution and potential data quality issues within the dataset. Below is a detailed analysis of the key statistical summaries:

**Central Tendency and Dispersion**
1. **Base Flood Elevation:**
- Average (Mean): 119.47 ft.
- Standard Deviation: 522.49 ft.
- Range: -9999 - 85,640 ft.
2. **Lowest Adjacent Grade:**
- Average (Mean): 129.20 ft.
- Standard Deviation: 609.92 ft.
- Range: -9,999 - 99,990.9 ft.
3. **Lowest Floor Elevation:**
- Average (Mean): 385.62 ft.
- Standard Deviation: 1,676.42 ft.
- Range: -9,997.9 - 99,989 ft.
4. **Basement Enclosure Crawl Space Type:**
- Average (Mean): 0.37, indicating a slight bias towards lower classifications.
- Standard Deviation: 0.86, showing moderate variability within the data.
- Range: Min 0 to Max 4, spanning several classification levels.
5. **Census Tract:**
- Average (Mean): Approximately 2.6 x 10¹⁰.
- Standard Deviation: About 1.58 x 10¹⁰, suggesting a wide spread across census tracts.
6. **CRS Discount:**
- Average: 0.064, typically low across the dataset.
- Standard Deviation: 0.091, with most data points close to zero but some higher values.
7. **Deductible Amount in Building and Contents Coverage:**
- Building Coverage Average: 1.66 with a deviation of 1.46.
- Contents Coverage Average: 0.98 with a deviation of 1.05.
- Both show low average deductible amounts but with notable variation.
8. **Elevation Difference:**
- Average: 1.69, indicating minor differences in elevation on average.
- Standard Deviation: 3.39, suggesting significant outliers affecting the elevation difference.
9. **Policy Related Figures (Policy Cost, Policy Count, Total Insurance Coverage, etc.):**
- These values have a high mean and standard deviation, indicating a significant spread in the policy costs and coverages, reflecting diverse insurance policies and property valuations.

**Extremes (Minimum and Maximum Values)**
- Notable minimums include negative values in **Federal Policy Fee and HFIAA Surcharge**, possibly indicating refunds or adjustments.
- The maximum values in **Total Building Insurance Coverage and Total Insurance Premium of the Policy** reach into the hundreds of millions, highlighting cases with exceptionally high insurance coverage.

**Implications**
The substantial missing data in critical geographical and elevation columns could significantly hinder risk assessment accuracy. The wide variability in policy costs and coverage levels underscores the diverse nature of the insured properties. Accurate and complete data in these fields are crucial for effective risk management and policy pricing in flood insurance.

This analysis provides a basis for further data cleaning, particularly in addressing missing values and outliers, which are essential for improving data quality and the reliability of subsequent analyses and decision-making processes based on this dataset.

### Correlations Among Variables

In [11]:
numerical_df.show(5,vertical=True)

-RECORD 0---------------------------------------
 basefloodelevation               | NULL        
 basementenclosurecrawlspacetype  | 2           
 censustract                      | 33013038500 
 countycode                       | 33013       
 crsdiscount                      | 0.0         
 elevationdifference              | 999         
 federalpolicyfee                 | 13          
 hfiaasurcharge                   | 0           
 latitude                         | 43.3        
 longitude                        | -71.8       
 lowestadjacentgrade              | NULL        
 lowestfloorelevation             | NULL        
 numberoffloorsininsuredbuilding  | 2           
 occupancytype                    | 1           
 policycost                       | 388         
 policycount                      | 1           
 policytermindicator              | 1           
 totalbuildinginsurancecoverage   | 250000      
 totalcontentsinsurancecoverage   | 100000      
 totalinsurancepremi

In [12]:
v_col = "features"
input = numerical_df.columns
output = [v_col + str(i) for i in range(len(input))]
imputer = Imputer(strategy='mean',inputCols=input,outputCols=output)
changed_df = imputer.fit(numerical_df).transform(numerical_df) 

assembler = VectorAssembler(inputCols=output,outputCol=v_col)
numerical_df_vector = assembler.transform(changed_df).select(v_col)
numerical_df_vector

DataFrame[features: vector]

In [13]:
matrix = Correlation.corr(numerical_df_vector,v_col).collect()[0][0]
corr_matrix = matrix.toArray().tolist()
print(corr_matrix)

24/05/24 13:41:12 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


[[1.0, 0.05835923233138655, 0.04534871506023088, 0.04452366797155514, -0.10226921108097828, 7.470058243085496e-05, -0.010387177126363122, 0.014284015128349724, 0.12638383851377266, -0.13551456752425814, 0.8181030501174288, 0.3047834007873046, 0.021232049918501958, 0.03948821384956352, -0.0017586364749227308, -0.008273477372144561, -1.6570593107843474e-05, -0.00797478163619458, -0.0343224403726591, -0.002348467821387104], [0.05835923233138655, 1.0, 0.13623903586845484, 0.1322243889691115, -0.13374568401426407, 0.04144637455805757, 0.021515636223077703, 0.039205129679828934, 0.4322749154690187, 0.10160449532374954, 0.050129140351338926, -0.014776239600981917, 0.4205917981132782, -0.06389760287441462, 0.07213744564440353, 0.00879790297900046, -0.024505685578106853, 0.00694688699877884, -0.09892408517021706, 0.0693259281247627], [0.04534871506023088, 0.13623903586845484, 1.0, 0.9861052821687085, -0.2363377259085023, 0.14064291796967068, -0.04651095355139787, -0.0017065025678084957, 0.27126

In [14]:
numerical_df.columns

['basefloodelevation',
 'basementenclosurecrawlspacetype',
 'censustract',
 'countycode',
 'crsdiscount',
 'elevationdifference',
 'federalpolicyfee',
 'hfiaasurcharge',
 'latitude',
 'longitude',
 'lowestadjacentgrade',
 'lowestfloorelevation',
 'numberoffloorsininsuredbuilding',
 'occupancytype',
 'policycost',
 'policycount',
 'policytermindicator',
 'totalbuildinginsurancecoverage',
 'totalcontentsinsurancecoverage',
 'totalinsurancepremiumofthepolicy']

In [15]:
columns = ['basefloodelevation',
 'basementenclosurecrawlspacetype',
 'censustract',
 'countycode',
 'crsdiscount',
 'elevationdifference',
 'federalpolicyfee',
 'hfiaasurcharge',
 'latitude',
 'longitude',
 'lowestadjacentgrade',
 'lowestfloorelevation',
 'numberoffloorsininsuredbuilding',
 'occupancytype',
 'policycost',
 'policycount',
 'policytermindicator',
 'totalbuildinginsurancecoverage',
 'totalcontentsinsurancecoverage',
 'totalinsurancepremiumofthepolicy']
df_c = spark.createDataFrame(corr_matrix,columns)

In [16]:
df_c.select('basefloodelevation',
 'basementenclosurecrawlspacetype',
 'censustract').show(20)

+--------------------+-------------------------------+--------------------+
|  basefloodelevation|basementenclosurecrawlspacetype|         censustract|
+--------------------+-------------------------------+--------------------+
|                 1.0|            0.05835923233138655| 0.04534871506023088|
| 0.05835923233138655|                            1.0| 0.13623903586845484|
| 0.04534871506023088|            0.13623903586845484|                 1.0|
| 0.04452366797155514|             0.1322243889691115|  0.9861052821687085|
|-0.10226921108097828|           -0.13374568401426407| -0.2363377259085023|
|7.470058243085496E-5|            0.04144637455805757| 0.14064291796967068|
|-0.01038717712636...|           0.021515636223077703|-0.04651095355139787|
|0.014284015128349724|           0.039205129679828934|-0.00170650256780...|
| 0.12638383851377266|             0.4322749154690187|  0.2712693913748728|
|-0.13551456752425814|            0.10160449532374954| 0.03869302136198077|
|  0.8181030

In [17]:
df_c.select('countycode',
 'crsdiscount',
 'elevationdifference').show(20)

+--------------------+--------------------+--------------------+
|          countycode|         crsdiscount| elevationdifference|
+--------------------+--------------------+--------------------+
| 0.04452366797155514|-0.10226921108097828|7.470058243085496E-5|
|  0.1322243889691115|-0.13374568401426407| 0.04144637455805757|
|  0.9861052821687085| -0.2363377259085023| 0.14064291796967068|
|                 1.0|-0.23953380471436225| 0.13888862687868733|
|-0.23953380471436225|                 1.0| -0.5259678054689083|
| 0.13888862687868733| -0.5259678054689083|                 1.0|
|-0.04591061242264458| 0.12618095833291212|-0.12308129153346879|
|-0.00380940314362...|0.048954413784206216|-0.04971959541878701|
|  0.2508501234655084|-0.27508558815552225| 0.19599408032117985|
|0.049632406252027084| 0.12376428198089964|-0.14955971097407328|
| 0.03410361028464174|-0.08582096552979489|0.002425271072278...|
|-6.80664449099432...|-0.00575452231836...|0.005525565608556745|
| 0.12667043463679634|-0.

In [18]:
df_c.select('federalpolicyfee',
 'hfiaasurcharge',
 'latitude').show(20)

+--------------------+--------------------+--------------------+
|    federalpolicyfee|      hfiaasurcharge|            latitude|
+--------------------+--------------------+--------------------+
|-0.01038717712636...|0.014284015128349724| 0.12638383851377266|
|0.021515636223077703|0.039205129679828934|  0.4322749154690187|
|-0.04651095355139787|-0.00170650256780...|  0.2712693913748728|
|-0.04591061242264458|-0.00380940314362...|  0.2508501234655084|
| 0.12618095833291212|0.048954413784206216|-0.27508558815552225|
|-0.12308129153346879|-0.04971959541878701| 0.19599408032117985|
|                 1.0| 0.15998037490116276|-0.02569128904685...|
| 0.15998037490116276|                 1.0| 0.04994652724067962|
|-0.02569128904685...| 0.04994652724067962|                 1.0|
| 0.03643340484637455|0.011012441102289104|-0.10822625068224143|
|-0.00935471628409477|0.011091913551252466| 0.10842310185825517|
|-0.01125795805632...|-0.01874631398887...| 0.01787552000870374|
| 0.10729663798512859|0.0

In [19]:
df_c.select('longitude',
 'lowestadjacentgrade',
 'lowestfloorelevation').show(20)

+--------------------+--------------------+--------------------+
|           longitude| lowestadjacentgrade|lowestfloorelevation|
+--------------------+--------------------+--------------------+
|-0.13551456752425814|  0.8181030501174288|  0.3047834007873046|
| 0.10160449532374954|0.050129140351338926|-0.01477623960098...|
| 0.03869302136198077| 0.03463261496948558|1.374835906710874...|
|0.049632406252027084| 0.03410361028464174|-6.80664449099432...|
| 0.12376428198089964|-0.08582096552979489|-0.00575452231836...|
|-0.14955971097407328|0.002425271072278...|0.005525565608556745|
| 0.03643340484637455|-0.00935471628409477|-0.01125795805632...|
|0.011012441102289104|0.011091913551252466|-0.01874631398887...|
|-0.10822625068224143| 0.10842310185825517| 0.01787552000870374|
|                 1.0|-0.12337700047640358|-0.04205996452248668|
|-0.12337700047640358|                 1.0|  0.2693569000679416|
|-0.04205996452248668|  0.2693569000679416|                 1.0|
| 0.12763844942700334|0.0

In [20]:
df_c.select('numberoffloorsininsuredbuilding',
 'occupancytype',
 'policycost').show(20)

+-------------------------------+--------------------+--------------------+
|numberoffloorsininsuredbuilding|       occupancytype|          policycost|
+-------------------------------+--------------------+--------------------+
|           0.021232049918501958| 0.03948821384956352|-0.00175863647492...|
|             0.4205917981132782|-0.06389760287441462| 0.07213744564440353|
|            0.13092070392149213| -0.0215185447700448|-0.00940672579516...|
|            0.12667043463679634|-0.02083336431546...|-0.01020270930151...|
|           -0.06978076159056372| 0.04701164748485697| 0.06739894183937774|
|           -0.02959368105409...|-0.05068082979967146|6.468266631659088E-4|
|            0.10729663798512859| 0.14237367075146548| 0.38647213373301115|
|           0.056225567874606905| 0.30145575938946867|  0.1823519286217399|
|            0.34375988157815457| 0.03707897744013729| 0.06080599667465571|
|            0.12763844942700334|-0.00248049165686...|0.025262116267291728|
|           

In [21]:
df_c.select('policycount',
 'policytermindicator',
 'totalbuildinginsurancecoverage').show(20)

+--------------------+--------------------+------------------------------+
|         policycount| policytermindicator|totalbuildinginsurancecoverage|
+--------------------+--------------------+------------------------------+
|-0.00827347737214...|-1.65705931078434...|          -0.00797478163619458|
| 0.00879790297900046|-0.02450568557810...|           0.00694688699877884|
|-0.02354252110006...| 0.03562507918662196|          -0.02664276057268567|
|-0.02335026782739...|0.041369073480967196|          -0.02736003235722...|
| 0.02814710877178117|-0.03970568654901237|          0.025212129977885484|
|-0.03065010725766519| 0.04104422570965058|          -0.03866066385828042|
|  0.5481074038104508|-0.02839057493420...|            0.4963566086681792|
|  0.0319498731974377|-0.02571907189115425|           0.03871098206689112|
|-0.02512727932134053|0.001390069725837...|          -0.02446116456239...|
|0.010198719294559726|0.005318202524779...|          0.007148866689062663|
|-0.00739083457659...|-1.

In [22]:
df_c.select('totalcontentsinsurancecoverage',
 'totalinsurancepremiumofthepolicy').show(20)

+------------------------------+--------------------------------+
|totalcontentsinsurancecoverage|totalinsurancepremiumofthepolicy|
+------------------------------+--------------------------------+
|           -0.0343224403726591|            -0.00234846782138...|
|          -0.09892408517021706|              0.0693259281247627|
|          0.014045383330901506|            -0.00788805859775...|
|          0.008361075386161178|            -0.00851191616326...|
|           -0.1443948051727502|             0.05863870325056883|
|           0.11507960281910591|            0.009922795564581026|
|          -0.09552873635534823|              0.3359488202322962|
|          0.014049042681092585|             0.11148432510815016|
|          -0.10442024470081457|             0.06066938471879821|
|          -0.00173734949061...|            0.023512821648468792|
|          -0.03205674352377111|            -0.00295009384701...|
|          -0.00959618414729...|            -0.01006926417078...|
|         

The dataframe new_df is the dataframe with the imputed mean values and this code checks that there are zero missing values and the column names are changed from feature0, feature1 and so on to the original column names within the new_df dataframe.

In [23]:
new_df = changed_df.drop('basefloodelevation',
 'basementenclosurecrawlspacetype',
 'censustract',
 'countycode',
 'crsdiscount',
 'elevationdifference',
 'federalpolicyfee',
 'hfiaasurcharge',
 'latitude',
 'longitude',
 'lowestadjacentgrade',
 'lowestfloorelevation',
 'numberoffloorsininsuredbuilding',
 'occupancytype',
 'policycost',
 'policycount',
 'policytermindicator',
 'totalbuildinginsurancecoverage',
 'totalcontentsinsurancecoverage',
 'totalinsurancepremiumofthepolicy')
new_df.show(5,vertical=True)

-RECORD 0------------------------
 features0  | 119.46875184553842 
 features1  | 2                  
 features2  | 33013038500        
 features3  | 33013              
 features4  | 0.0                
 features5  | 999                
 features6  | 13                 
 features7  | 0                  
 features8  | 43.3               
 features9  | -71.8              
 features10 | 129.1970600006573  
 features11 | 385.6220114835789  
 features12 | 2                  
 features13 | 1                  
 features14 | 388                
 features15 | 1                  
 features16 | 1                  
 features17 | 250000             
 features18 | 100000             
 features19 | 375                
-RECORD 1------------------------
 features0  | 119.46875184553842 
 features1  | 0                  
 features2  | 22063040700        
 features3  | 22063              
 features4  | 0.05               
 features5  | 999                
 features6  | 35                 
 features7  | 

In [24]:
new_df_missing_vals = new_df.select(*(spark_sum(col(i).isNull().cast("int")).alias(i) for i in new_df.columns))

In [25]:
new_df_missing_vals.show(vertical= True)

-RECORD 0---------
 features0  | 0   
 features1  | 0   
 features2  | 0   
 features3  | 0   
 features4  | 0   
 features5  | 0   
 features6  | 0   
 features7  | 0   
 features8  | 0   
 features9  | 0   
 features10 | 0   
 features11 | 0   
 features12 | 0   
 features13 | 0   
 features14 | 0   
 features15 | 0   
 features16 | 0   
 features17 | 0   
 features18 | 0   
 features19 | 0   



In [26]:
column_names = ['basefloodelevation',
 'basementenclosurecrawlspacetype',
 'censustract',
 'countycode',
 'crsdiscount',
 'elevationdifference',
 'federalpolicyfee',
 'hfiaasurcharge',
 'latitude',
 'longitude',
 'lowestadjacentgrade',
 'lowestfloorelevation',
 'numberoffloorsininsuredbuilding',
 'occupancytype',
 'policycost',
 'policycount',
 'policytermindicator',
 'totalbuildinginsurancecoverage',
 'totalcontentsinsurancecoverage',
 'totalinsurancepremiumofthepolicy']
new_df = new_df.toDF(*column_names)
new_df.show(5,vertical=True)

-RECORD 0----------------------------------------------
 basefloodelevation               | 119.46875184553842 
 basementenclosurecrawlspacetype  | 2                  
 censustract                      | 33013038500        
 countycode                       | 33013              
 crsdiscount                      | 0.0                
 elevationdifference              | 999                
 federalpolicyfee                 | 13                 
 hfiaasurcharge                   | 0                  
 latitude                         | 43.3               
 longitude                        | -71.8              
 lowestadjacentgrade              | 129.1970600006573  
 lowestfloorelevation             | 385.6220114835789  
 numberoffloorsininsuredbuilding  | 2                  
 occupancytype                    | 1                  
 policycost                       | 388                
 policycount                      | 1                  
 policytermindicator              | 1           

In [27]:
print(new_df.dtypes)

[('basefloodelevation', 'double'), ('basementenclosurecrawlspacetype', 'int'), ('censustract', 'bigint'), ('countycode', 'int'), ('crsdiscount', 'double'), ('elevationdifference', 'int'), ('federalpolicyfee', 'int'), ('hfiaasurcharge', 'int'), ('latitude', 'double'), ('longitude', 'double'), ('lowestadjacentgrade', 'double'), ('lowestfloorelevation', 'double'), ('numberoffloorsininsuredbuilding', 'int'), ('occupancytype', 'int'), ('policycost', 'int'), ('policycount', 'int'), ('policytermindicator', 'int'), ('totalbuildinginsurancecoverage', 'int'), ('totalcontentsinsurancecoverage', 'int'), ('totalinsurancepremiumofthepolicy', 'int')]


In [ ]:
# Milestone 3: Construct Linear Regression model to perform Feature Importance (find weights of features, choose top weights' features to keep)
# creating train and test sets
# X = new_df.drop('totalinsurancepremiumofthepolicy')
# y = df['totalinsurancepremiumofthepolicy']
# ASK: Is 'features' correct?
assembler = VectorAssembler(
    inputCols=['basefloodelevation',
 'basementenclosurecrawlspacetype',
 'censustract',
 'countycode',
 'crsdiscount',
 'elevationdifference',
 'federalpolicyfee',
 'hfiaasurcharge',
 'latitude',
 'longitude',
 'lowestadjacentgrade',
 'lowestfloorelevation',
 'numberoffloorsininsuredbuilding',
 'occupancytype',
 'policycost',
 'policycount',
 'policytermindicator',
 'totalbuildinginsurancecoverage',
 'totalcontentsinsurancecoverage'],
    outputCol = 'features')
new_df = assembler.transform(new_df)
final_data = new_df.select('features', 'totalinsurancepremiumofthepolicy')

train_data, test_data = final_data.randomSplit([0.7, 0.3], seed=42)

lr = LinearRegression(featuresCol = 'features', labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium', regParam = 0.3)
lr_model = lr.fit(train_data)

In [ ]:
# Milestone 3: Print coefficients of the features (will run this cell AFTER LinReg is successful)
coefficients = lr_model.coefficients
print("Coefficients: {}".format(lr_model.coefficients)) 

In [ ]:
# Milestone 3: Construct Feature Importance to find top 10 features to choose to represent our dataset.
feature_importance = sorted(list(zip(new_df.columns[:-1], map(abs, coefficients))), key=lambda x: x[1], reverse=True)

print("Feature Importance:")
for feature, importance in feature_importance:
    print("  {}: {:.3f}".format(feature, importance))

# Construct 5 Linear Regression Models to find Ideal Range for Model Complexity
#### Using the Feature Importance model information, will start by training a Linear Regression Model with 1 feature (Top Feature Importance), then 5 features, 10 features, 15 features, and finally 19 features. There are a total 20 numerical features in the FEMA dataset, and the totalinsurancepremiumofthepolicy feature is the dependent variable. 

### Linear Regression Model with 1 Feature: crsdiscount

In [ ]:
assembler_f1 = VectorAssembler(
    inputCols=['crsdiscount'], outputCol = 'predict_features_1')

new_df = assembler_f1.transform(new_df)

df_f1 = new_df.select('predict_features_1', 'totalinsurancepremiumofthepolicy')
train_data_f1, test_data_f1 = df_f1.randomSplit([0.7, 0.3], seed = 42)

lr_f1 = LinearRegression(featuresCol = 'predict_features_1', labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f1', regParam = 0.3)
lr_model_f1 = lr_f1.fit(train_data_f1)

In [ ]:
# RMSE for Train data
predictions_rmse_train_f1 = lr_model_f1.transform(train_data_f1)

evaluator_f1 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f1', metricName = 'rmse')
rmse_train_f1 = evaluator.evaluate(predictions_rmse_train_f1)
print("Root Mean Squared Error (RMSE) on train data for Linear Regression using 1 feature: {:.7f}".format(rmse_train_f1))

In [ ]:
# R-squared for Train data
evaluator_r2_f1 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f1', metricName = 'r2')
r2_train_f1 = evaluator_r2_f1.evaluate(predictions_rmse_train_f1)
print("R-squared (R2) on train data for Linear Regression using 1 feature: {:.7f}".format(r2_train_f1))

In [ ]:
# RMSE for Test data
predictions_rmse_test_f1 = lr_model_2.transform(test_data_f1)

evaluator_f1 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f1', metricName = 'rmse')
rmse_test_f1 = evaluator.evaluate(predictions_rmse_test_f1)
print("Root Mean Squared Error (RMSE) on test data for Linear Regression using 1 feature: {:.7f}".format(rmse_test_f1))

In [ ]:
# R-squared for Test data
evaluator_r2_f1 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f1', metricName = 'r2')
r2_test_f1 = evaluator_r2_f1.evaluate(predictions_rmse_test_f1)
print("R-squared (R2) on test data for Linear Regression using 1 feature: {:.7f}".format(r2_test_f1))

### Linear Regression Model with 5 Features: 
#### crsdiscount', 'policytermindicator', 'occupancytype', 'policycount', basementenclosurecrawlspacetype'

In [ ]:
assembler_f5 = VectorAssembler(
    inputCols=['crsdiscount',
  'policytermindicator',
  'occupancytype',
  'policycount',
  'basementenclosurecrawlspacetype'], outputCol = 'predict_features_5')

new_df = assembler_f5.transform(new_df)

df_f5 = new_df.select('predict_features_5', 'totalinsurancepremiumofthepolicy')
train_data_f5, test_data_f5 = df_f5.randomSplit([0.7, 0.3], seed = 42)

lr_f5 = LinearRegression(featuresCol = 'predict_features_5', labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f5', regParam = 0.3)
lr_model_f5 = lr_f5.fit(train_data_f5)

In [ ]:
# RMSE for Train data
predictions_rmse_train_f5 = lr_model_f5.transform(train_data_f5)

evaluator_f5 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f5', metricName = 'rmse')
rmse_train_f5 = evaluator.evaluate(predictions_rmse_train_f5)
print("Root Mean Squared Error (RMSE) on train data for Linear Regression using 5 features: {:.7f}".format(rmse_train_f5))

In [ ]:
# R-squared for Train data
evaluator_r2_f5 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f5', metricName = 'r2')
r2_train_f5 = evaluator_r2_f5.evaluate(predictions_rmse_train_f5)
print("R-squared (R2) on train data for Linear Regression using 5 features: {:.7f}".format(r2_train_f5))

In [ ]:
# RMSE for Test data
predictions_rmse_test_f5 = lr_model_2.transform(test_data_f5)

evaluator_f5 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f5', metricName = 'rmse')
rmse_test_f5 = evaluator.evaluate(predictions_rmse_test_f5)
print("Root Mean Squared Error (RMSE) on test data for Linear Regression using 5 features: {:.7f}".format(rmse_test_f5))

In [ ]:
# R-squared for Test data
evaluator_r2_f5 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f5', metricName = 'r2')
r2_test_f5 = evaluator_r2_f5.evaluate(predictions_rmse_test_f5)
print("R-squared (R2) on test data for Linear Regression using 5 features: {:.7f}".format(r2_test_f5))

### Linear Regression Model with 10 Features:
#### 'basementenclosurecrawlspacetype', 'crsdiscount', 'federalpolicyfee', 'hfiaasurcharge', 'latitude', 'numberoffloorsininsuredbuilding', 'occupancytype', 'policycost', 'policycount', 'policytermindicator'

In [ ]:
# Milestone 3: Construct Linear Regression prediction model with the new Chosen 10 Features
# f10 = LinReg model with top 10 features (chosen from Feature Importance)

assembler_f10 = VectorAssembler(
    inputCols=['basementenclosurecrawlspacetype',
 'crsdiscount',
 'federalpolicyfee',
 'hfiaasurcharge',
 'latitude',
 'numberoffloorsininsuredbuilding',
 'occupancytype',
 'policycost',
 'policycount',
 'policytermindicator'],
    outputCol = 'predict_features_10')

new_df = assembler_f10.transform(new_df)

df_f10 = new_df.select('predict_features_10', 'totalinsurancepremiumofthepolicy')
train_data_f10, test_data_f10 = df_f10.randomSplit([0.7, 0.3], seed = 42)

lr_f10 = LinearRegression(featuresCol = 'predict_features_10', labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f10', regParam = 0.3)
lr_model_f10 = lr_f10.fit(train_data_f10)

In [ ]:
# Milestone 3: RMSE for Train data
# RMSE ranges from 0 to infinity, lower RMSE the better, (higher weight to larger errors)
predictions_rmse_train_f10 = lr_model_f10.transform(train_data_f10)

evaluator_f10 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f10', metricName = 'rmse')
rmse_train_f10 = evaluator.evaluate(predictions_rmse_train)
print("Root Mean Squared Error (RMSE) on train data for Linear Regression using 10 features: {:.7f}".format(rmse_train_f10))

In [ ]:
# R-squared for Train data
evaluator_r2_f10 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f10', metricName = 'r2')
r2_train_f10 = evaluator_r2_f10.evaluate(predictions_rmse_train_f10)
print("R-squared (R2) on train data for Linear Regression using 10 features: {:.7f}".format(r2_train_f10))

In [ ]:
# RMSE for Test data
predictions_rmse_test_f10 = lr_model_2.transform(test_data_f10)

evaluator_f10 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f10', metricName = 'rmse')
rmse_test_f10 = evaluator.evaluate(predictions_rmse_test_f10)
print("Root Mean Squared Error (RMSE) on test data for Linear Regression using 10 features: {:.7f}".format(rmse_test_f10))

In [ ]:
# R-squared for Test data
evaluator_r2_f10 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f10', metricName = 'r2')
r2_test_f10 = evaluator_r2_f10.evaluate(predictions_rmse_test_f10)
print("R-squared (R2) on test data for Linear Regression using 10 features: {:.7f}".format(r2_test_f10))

### Linear Regression Model with 15 Features
#### 'crsdiscount', 'policytermindicator', 'occupancytype', 'policycount', 'basementenclosurecrawlspacetype', 'hfiaasurcharge', 'federalpolicyfee', 'policycost', 'numberoffloorsininsuredbuilding', 'latitude', 'longitude', 'elevationdifference', 'lowestadjacentgrade', 'lowestfloorelevation', 'basefloodelevation'

In [ ]:
assembler_f15 = VectorAssembler(
    inputCols=['crsdiscount',
  'policytermindicator',
  'occupancytype',
  'policycount',
  'basementenclosurecrawlspacetype',
  'hfiaasurcharge',
  'federalpolicyfee',
  'policycost',
  'numberoffloorsininsuredbuilding',
  'latitude',
  'longitude',
  'elevationdifference',
  'lowestadjacentgrade',
  'lowestfloorelevation',
  'basefloodelevation'], outputCol = 'predict_features_15')

new_df = assembler_f15.transform(new_df)

df_f15 = new_df.select('predict_features_15', 'totalinsurancepremiumofthepolicy')
train_data_f15, test_data_f15 = df_f15.randomSplit([0.7, 0.3], seed = 42)

lr_f15 = LinearRegression(featuresCol = 'predict_features_15', labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f15', regParam = 0.3)
lr_model_f15 = lr_f15.fit(train_data_f15)

In [ ]:
# RMSE for Train data
predictions_rmse_train_f15 = lr_model_f15.transform(train_data_f15)

evaluator_f15 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f15', metricName = 'rmse')
rmse_train_f15 = evaluator.evaluate(predictions_rmse_train_f15)
print("Root Mean Squared Error (RMSE) on train data for Linear Regression using 15 features: {:.7f}".format(rmse_train_f15))

In [ ]:
# R-squared for Train data
evaluator_r2_f15 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f15', metricName = 'r2')
r2_train_f15 = evaluator_r2_f15.evaluate(predictions_rmse_train_f15)
print("R-squared (R2) on train data for Linear Regression using 15 features: {:.7f}".format(r2_train_f15))

In [ ]:
# RMSE for Test data
predictions_rmse_test_f15 = lr_model_2.transform(test_data_f15)

evaluator_f15 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f15', metricName = 'rmse')
rmse_test_f15 = evaluator.evaluate(predictions_rmse_test_f15)
print("Root Mean Squared Error (RMSE) on test data for Linear Regression using 15 features: {:.7f}".format(rmse_test_f15))

In [ ]:
# R-squared for Test data
evaluator_r2_f15 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f15', metricName = 'r2')
r2_test_f15 = evaluator_r2_f15.evaluate(predictions_rmse_test_f15)
print("R-squared (R2) on test data for Linear Regression using 15 features: {:.7f}".format(r2_test_f15))

### Linear Regression Model with 19 Features
#### 'crsdiscount', 'policytermindicator', 'occupancytype', 'policycount', 'basementenclosurecrawlspacetype', 'hfiaasurcharge', 'federalpolicyfee', 'policycost', 'numberoffloorsininsuredbuilding', 'latitude', 'longitude', 'elevationdifference', 'lowestadjacentgrade:', 'lowestfloorelevation', 'basefloodelevation', 'countycode', 'totalbuildinginsurancecoverage', 'totalcontentsinsurancecoverage', 'censustract'

In [ ]:
assembler_f19 = VectorAssembler(
    inputCols = ['crsdiscount',
  'policytermindicator',
  'occupancytype',
  'policycount',
  'basementenclosurecrawlspacetype',
  'hfiaasurcharge',
  'federalpolicyfee',
  'policycost',
  'numberoffloorsininsuredbuilding',
  'latitude',
  'longitude',
  'elevationdifference',
  'lowestadjacentgrade:',
  'lowestfloorelevation',
  'basefloodelevation',
  'countycode',
  'totalbuildinginsurancecoverage',
  'totalcontentsinsurancecoverage',
  'censustract'], outputCol = 'predict_features_19')

new_df = assembler_f19.transform(new_df)

df_f19 = new_df.select('predict_features_15', 'totalinsurancepremiumofthepolicy')
train_data_f19, test_data_f19 = df_f19.randomSplit([0.7, 0.3], seed = 42)

lr_f19 = LinearRegression(featuresCol = 'predict_features_19', labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f19', regParam = 0.3)
lr_model_f19 = lr_f19.fit(train_data_f19)

In [ ]:
# RMSE for Train data
predictions_rmse_train_f19 = lr_model_f19.transform(train_data_f19)

evaluator_f19 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f19', metricName = 'rmse')
rmse_train_f19 = evaluator.evaluate(predictions_rmse_train_f19)
print("Root Mean Squared Error (RMSE) on train data for Linear Regression using 19 features: {:.7f}".format(rmse_train_f19))

In [ ]:
# R-squared for Train data
evaluator_r2_f19 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f19', metricName = 'r2')
r2_train_f19 = evaluator_r2_f19.evaluate(predictions_rmse_train_f19)
print("R-squared (R2) on train data for Linear Regression using 19 features: {:.7f}".format(r2_train_f19))

In [ ]:
# RMSE for Test data
predictions_rmse_test_f19 = lr_model_2.transform(test_data_f19)

evaluator_f19 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f19', metricName = 'rmse')
rmse_test_f19 = evaluator.evaluate(predictions_rmse_test_f19)
print("Root Mean Squared Error (RMSE) on test data for Linear Regression using 19 features: {:.7f}".format(rmse_test_f19))

In [ ]:
# R-squared for Test data
evaluator_r2_f19 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f19', metricName = 'r2')
r2_test_f19 = evaluator_r2_f19.evaluate(predictions_rmse_test_f19)
print("R-squared (R2) on test data for Linear Regression using 19 features: {:.7f}".format(r2_test_f19))

## Note to group: exclude Bar plots when submitting for Milestone 4. This is incorrect; not a Fitting Graph.

In [ ]:
# Milestone 3: Bar plot with RMSE values, comparing train and test data
# To answer Question 4: "Where does your model fit in the fitting graph?"
df_barplot = pd.DataFrame({'RMSE': ['Train RMSE', 'Test RMSE'], 'Error Values': [118.133, 117.118]})  
df_barplot.plot.bar(x = 'RMSE', y = 'Error Values')  

In [ ]:
# Milestone 3: Bar plot with R-squared values, comparing train and test data
# To answer Question 4: "Where does your model fit in the fitting graph?"
df_barplot = pd.DataFrame({'R-Squared': ['Train R-Squared', 'Test R-Squared'], 'Values': [0.995, 0.995]})  
df_barplot.plot.bar(x = 'R-Squared', y = 'Values')  

### Scatterplots 

In [ ]:
changed_num_df = changed_df.sample(False, 0.1)
num_pdf = changed_num_df.toPandas()
print(num_pdf)

In [ ]:
plt.scatter(x=num_pdf['policycost'],y=num_pdf['totalinsurancepremiumofthepolicy'])
plt.xlabel('Policy Cost')
plt.ylabel('Total Insurance Premium of the Policy')          
plt.title('Policy Cost vs Total Insurance Premium of the Policy')
plt.show()

In [ ]:
plt.scatter(x=num_pdf['countycode'],y=num_pdf['censustract'])
plt.xlabel('County Code')
plt.ylabel('Census Tract') 
plt.title('County Code vs Census Tract')
plt.show()

In [ ]:
plt.scatter(x=num_pdf['policycount'],y=num_pdf['totalbuildinginsurancecoverage'])
plt.xlabel('Policy Count')
plt.ylabel('Total Building Insurance Coverage') 
plt.title('Policy Count vs Total Building Insurance Coverage')
plt.show()